<a href="https://colab.research.google.com/github/hanna-joo/statistics/blob/master/stat_python/ch_10_ANOVA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10장. 분산분석 (ANOVA: Analysis of Variance)
part 01
  - 10.1. 분산분석 기초
  - 10.2. 일원분산분석
  - 10.3. 이원분산분석
  ___
part 02
  - 10.4. 다변량분산분석
  - 10.5. 공분산분석

## 10.0. 준비하기

In [ ]:
# 구글 드라이브 연동
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)       
          # print content of ROOT (Optional)
drive.mount(ROOT, force_remount=True)

/content/drive
Mounted at /content/drive


In [ ]:
from os.path import join  

MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks/python_stat/python_stat_data/pythondata'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

%cd "{PROJECT_PATH}"
%ls

/content/drive/My Drive/Colab Notebooks/python_stat/python_stat_data/pythondata
'실습파일_10장 분산분석.ipynb'
'실습파일_11장 회귀분석.ipynb'
'실습파일_12장 요인분석.ipynb'
'실습파일_13장 분류예측분석.ipynb'
'실습파일_14장 군집분석.ipynb'
'실습파일_15장 포지셔닝 분석.ipynb'
'실습파일_16장 컨조인트 분석.ipynb'
'실습파일_17장 비모수 통계분석.ipynb'
'실습파일_3장 파이썬 기초.ipynb'
'실습파일_4장 데이터전처리 및 기초분석.ipynb'
'실습파일_5장 기술통계분석.ipynb'
'실습파일_6장 t 검정.ipynb'
'실습파일_7장 상관관계분석.ipynb'
'실습파일_8장 범주형 데이터 분석.ipynb'
'실습파일_9장 신뢰성 분석.ipynb'
 Ashopping.csv
 CCA.csv
 Conjoint.csv
 Correspondence.csv
 부록_통계표.docx
 MDS1.csv
 MDS3.csv
'Step by Step 파이썬 비즈니스 통계분석_정오표.xlsx'


In [ ]:
# 분산분석 패키지 설치
!pip install scikit_posthocs # 다양한 사후분석 기능 제공
!pip install pingouin

In [ ]:
# 모듈 및 데이터 탑재
import pandas as pd
import numpy as np
import scipy as sp
import pingouin as pg
import scikit_posthocs

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('Ashopping.csv', sep=',', encoding='CP949')

## 10.1. 분산분석 기초
- 독립변수: 명목척도
- 종속변수: 등간/비율척도
- 3개 이상 집단간 평균의 차이로 독립변수와 종속변수 사이의 관계를 검정

### (1) 분산분석의 종류
|    |분산분석 종류|독립변수|종속변수|예시|
|:----|:------|:---|:---|:---|
|단일변량|**일원분산분석**|1개|1개|고객등급별 매장 방문횟수의 비교|
|단일변량|**이원분산분석**|2개|1개|성별, 나이대별 통신요금의 차이 비교|
|다변량|**다변량분산분석**|2개|2개이상|성별, 가격에 따른 판촉 전 판매량과 판촉 후 판매량 차이 비교|

> * **공분산분석**
  - 연속형 외생변수가 종속변수에 미치는 영향 제거 > 순수한 집단간 종속변수의 평균 차이 비교  
  - 예시: 고객등급별 매장 방문횟수의 비교 시 고객 나이를 
통제한 상황에서 분석

> * **사후분석**
  - 분산분석 후 구체적으로 어떤 집단이 어떤 집단과 유의한 차이를 보이는지 비교할 수 있는 방법
  - 방법:
    + **Fisher 의 최소유의차**: 등분산 가정하는 방법 / 주로 귀무가설 기각되는 경우에 사용하며 각 집단의 표본 크기가 다른 경우에도 적용 가능
    + **Tukey 의 정직유의차**: 위 방법보다 엄격한 방법 /  일반적으로 검정력이 떨어지기 때문에 보통 유의수준을 0.05가 아닌 0.1 이상의 큰 값으로 분석 / 집단간 차이를 가장 정밀하게 감지할 수 있지만 집단별 표본 크기가 같은 경우에만 의미가 있음
    + **Scheffe 방법**: 일반적이고 융통성 있는 방법 / 집단별 표본 크기가 다른 경우에도 가능 / 필요이상으로 넓은 신뢰구간 제시하는 단점
    + **Duncan 방법**: Fisher의 최소유의차와 마찬가지로 등분산 가정 / 1종 오류를 범할 가능성이 높다는 단점






## 10.2. 일원분산분석 (One-way ANOVA)
### (1) 개념 및 주요 이론
- 분산분석에서 가설검정을 위해 사용되는 검정통계량은 F값
  + F값
    + 집단내 분산 대비 집단간 분산이 몇 배 더 큰지
    + (F 임계치) < (F값): 집단 간의 차이가 충분히 크다
  + 집단내 분산(Sum of Square Between Groups) 
    + 각 집단의 평균치를 중심으로 각 집단내의 자료들이 우연적인 오차에 의해 어떻게 흩어져 있는가를 요약하는 척도
  + 집단간 분산(Sum of Square Within Groups)
    + 각 집단의 평균들이 전체 평균으로부터 흩어진 정도를 나타내는 척도
  + 총 분산(Sum of Square Total)
    + 각 자료들이 전체 평균으로부터 흩어진 정도를 나타내는 척도

### (2) 분석 및 해석
- 문제
  + A 쇼핑 고객들의 구매유형별 총 매출액의 차이가 있는지를 일원분산분석을 통해 검정해 보자.  
A 쇼핑에서 관리하는 고객들의 구매유형과 고객 수는 다음과 같다.

|코드|구매유형|고객 수(명)|
|:----:|:------:|:---:|
|1|1회성 구매형|43|
|2|실용적 구매형|317|
|3|명품 구매형|144|
|4|집중 구매형|496|

- 가설 
  + H0(귀무가설) = A 쇼핑 고객의 구매유형에 따른 총 매출액의 차이는 없다.
  + H1(연구가설) = A 쇼핑 고객은 적어도 1개의 구매유형이 다른 구매유형과 총 매출액 차이가 존재한다.

In [ ]:
# 0. 모듈 및 데이터 탑재
import pandas as pd
import numpy as np
import scipy as sp
import pingouin as pg
import scikit_posthocs

df = pd.read_csv('Ashopping.csv', sep=',', encoding='CP949')

In [ ]:
df.head()

,고객ID,이탈여부,총_매출액,방문빈도,1회_평균매출액,할인권_사용 횟수,총_할인_금액,고객등급,구매유형,클레임접수여부,구매_카테고리_수,거주지역,성별,고객_나이대,거래기간,할인민감여부,멤버쉽_프로그램_가입전_만족도,멤버쉽_프로그램_가입후_만족도,Recency,Frequency,Monetary,상품_만족도,매장_만족도,서비스_만족도,상품_품질,상품_다양성,가격_적절성,상품_진열_위치,상품_설명_표시,매장_청결성,공간_편의성,시야_확보성,음향_적절성,안내_표지판_설명,친절성,신속성,책임성,정확성,전문성
0,1,0,4007080,17,235711,1,5445,1,4,0,6,6,1,4,1079,0,5,7,7,3,4,6,5,6,7,7,6,7.0,6.0,6,7,6,6,6,6,6,6,6,6
1,2,1,3168400,14,226314,22,350995,2,4,0,4,4,1,1,537,0,2,3,2,3,3,2,5,4,6,7,6,6.0,NaN,7,7,6,6,6,5,3,6,6,6
2,3,0,2680780,18,148932,6,186045,1,4,1,6,6,1,6,1080,0,6,6,7,3,2,4,6,7,6,7,6,7.0,NaN,6,6,6,6,6,7,7,6,6,7
3,4,0,5946600,17,349800,1,5195,1,4,1,5,5,1,6,1019,0,3,5,7,3,5,3,5,5,6,6,6,5.0,6.0,6,6,5,6,6,6,6,6,5,6
4,5,0,13745950,73,188301,9,246350,1,2,0,6,6,0,6,1086,0,5,6,7,6,7,5,6,6,5,6,6,5.0,6.0,5,6,6,6,5,5,6,6,5,6


In [ ]:
df1 = df[['구매유형','총_매출액']]
pd.options.display.float_format = '{:.3f}'.format # df 내 수치형 데이터를 소수점 셋째 자리까지 출력

df1.head()

,구매유형,총_매출액
0,4,4007080
1,4,3168400
2,4,2680780
3,4,5946600
4,2,13745950


In [ ]:
# 1-1. 등분산 검정: 행렬 형태로 저장
구매유형 = []
for i in range(1,5,1):
  구매유형.append(df1[df1.구매유형==i].총_매출액)

In [ ]:
type(df1[df1.구매유형==1].총_매출액)

pandas.core.series.Series

In [ ]:
# 1-2. 등분산 검정: Bartlett/flinger/levene
  # Bartlett - 데이터가 정규분포인 경우
  # flinger - 비모수 검정으로써 정규성을 논하기 어려울 경우
  # levene - 데이터가 정규분포를 따르지 않을 경우 주로 사용
print("1. 등분산 검정\n")
sp.stats.levene(구매유형[0],구매유형[1],구매유형[2],구매유형[3])

1. 등분산 검정



LeveneResult(statistic=61.83834278363635, pvalue=1.1483869977419955e-36)

In [ ]:
# 2. Welch 일원분산분석 (등분산이 아닌 경우)
print("2. Welch 일원분산분석 (등분산이 아닌 경우)\n")
print(pg.welch_anova(dv='총_매출액',between='구매유형',data=df1))

print("*"*50)
# 3. 사후분석
  # 구매유형 데이터는 범주형 자료이기에 문자열로 변경
print("3. 사후분석\n")
df1['구매유형'] = df1['구매유형'].astype(str)
print(scikit_posthocs.posthoc_scheffe(df1, val_col='총_매출액',group_col='구매유형'))

print("*"*50)
# 4. 구매유형별 평균 총매출액
print("4. 구매유형별 평균 총매출액\n")
print(구매유형[0].mean(), 구매유형[1].mean(), 구매유형[2].mean(), 구매유형[3].mean())

2. Welch 일원분산분석 (등분산이 아닌 경우)

  Source  ddof1   ddof2      F  p-unc   np2
0   구매유형      3 230.936 88.238  0.000 0.193
**************************************************
3. 사후분석

      4     2     1     3
4 1.000 0.000 0.805 0.000
2 0.000 1.000 0.000 0.008
1 0.805 0.000 1.000 0.000
3 0.000 0.008 0.000 1.000
**************************************************
4. 구매유형별 평균 총매출액

3403682.3255813955 9612645.078864353 11779839.652777778 4392794.395161291


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


- 결과 해석
1. 등분산 검정 결과
  + F값: 61.83
  + 유의확률: 0.01 미만 > 등분산이 아니다(연구가설)
2. 유의확률 0.01 이하로 귀무가설 기각
3. 구매유형 1번과 4번을 제외한 나머지 구매유형 그룹간에 유의한 차이 존재
4. 명품 구매형 > 실용적 구매형 > 집중 구매형 > 1회성 구매형

  => 연구가설: A 쇼핑 고객은 적어도 1개의 구매유형이 다른 구매유형과 총 매출액 차이가 존재한다. (1번과 4번 제외)


## 10.3. 이원분산분석 (Two-way ANOVA)
### (1) 개념 및 주요 이론
- 독립변수 2개, 종속변수 1개
- 독립변수들 간의 상호작용 효과 추가로 판단
          상호작용
          - 종속변수에 대한 독립변수들의 결합효과 
          - 종속변수에 대한 한 독립변수의 효과가 다른 독립변수의 각 수준에서 동일하지 않다
  + 만약 상호작용 존재하지 않을 경우 2개의 독립변수에 대한 각각의 일원분산분석 수행하는 것과 동일


### (2) 분석 및 해석
- 문제
  + 구매유형과 거주지역에 따라 고객들의 총 매출액이 다른지 검정해보자.  
  이원분산분석의 가설은 제1 독립변수의 효과, 제2 독립변수의 효과,  
  그리고 상호작용 효과에 대해 논하는 가설로 분리하여 설정하는 것이 바람직하다.

|코드|구매유형|고객 수(명)|
|:----:|:------:|:---:|
|1|1회성 구매형|43|
|2|실용적 구매형|317|
|3|명품 구매형|144|
|4|집중 구매형|496|

- 가설
  - **제1 독립변수**의 효과: 구매유형에 따른 가설 
    + H0(귀무가설) = 구매유형에 따른 총 매출액의 차이는 없다.
    + H1(연구가설) = 적어도 한개의 구매유형이 다른 구매유형과 총 매출액 차이가 존재한다.
  - **제2 독립변수**의 효과: 거주지역에 따른 가설
    + H0(귀무가설) = 거주지역에 따른 총 매출액의 차이는 없다.
    + H1(연구가설) = 적어도 1개의 거주지역이 다른 거주지역과 총 매출액 차이가 존재한다.
  - **상호작용** 효과: 독립변수간 상호작용에 따른 가설
    + H0(귀무가설) = 구매유형과 거주지역의 상호작용 효과는 없다.
    + H1(연구가설) = 구매유형과 거주지역의 상호작용 효과가 있다.

In [ ]:
# 0. 모듈 및 데이터 탑재
import pandas as pd
import numpy as np
import scipy as sp
import pingouin as pg
import scikit_posthocs

df = pd.read_csv('Ashopping.csv', sep=',', encoding='CP949')
df1 = df[['총_매출액','구매유형','거주지역']]
pd.options.display.float_format = '{:.3f}'.format # df 내 수치형 데이터를 소수점 셋째 자리까지 출력

In [ ]:
# 1. 등분산성 검정

구매유형 = []
for i in range(1,5,1):
  구매유형.append(df1[df1.구매유형==i].총_매출액)

거주지역 = []
for i in range(1,8,1):
  거주지역.append(df1[df1.거주지역==i].총_매출액)

print("1. 등분산 검정\n")
print(sp.stats.levene(구매유형[0],구매유형[1],구매유형[2],구매유형[3]))
print(sp.stats.levene(거주지역[0],거주지역[1],거주지역[2],거주지역[3],거주지역[4],거주지역[5],거주지역[6]))

1. 등분산 검정

LeveneResult(statistic=61.83834278363635, pvalue=1.1483869977419955e-36)
LeveneResult(statistic=15.505555242738524, pvalue=4.643266523366773e-17)


In [ ]:
# 2. 이원분산분석
print(pg.anova(dv='총_매출액', between=['구매유형','거주지역'], data=df1))

        Source                    SS      DF  ...      F  p-unc   np2
0         구매유형  6318565682974362.000   3.000  ... 64.046  0.000 0.164
1         거주지역 16139868513734124.000   6.000  ... 81.798  0.000 0.335
2  구매유형 * 거주지역  3867936213048951.000  18.000  ...  6.534  0.000 0.108
3     Residual 32096439113329668.000 976.000  ...    nan    nan   nan

[4 rows x 7 columns]


In [ ]:
# 3. 사후분석
df1['구매유형'] = df1['구매유형'].astype(str)
df1['거주지역'] = df1['거주지역'].astype(str)
  # 각 집단별 표본 크기가 다르기 때문에 scheffe 방법 이용
print(scikit_posthocs.posthoc_scheffe(df1,val_col='총_매출액',group_col='구매유형'))
print(scikit_posthocs.posthoc_scheffe(df1,val_col='총_매출액',group_col='거주지역'))

      4     2     1     3
4 1.000 0.000 0.805 0.000
2 0.000 1.000 0.000 0.008
1 0.805 0.000 1.000 0.000
3 0.000 0.008 0.000 1.000
      6     4     5     7     3     2     1
6 1.000 0.008 0.002 0.000 0.019 0.339 0.988
4 0.008 1.000 1.000 0.000 0.978 0.980 1.000
5 0.002 1.000 1.000 0.000 0.935 0.965 1.000
7 0.000 0.000 0.000 1.000 0.000 0.000 0.412
3 0.019 0.978 0.935 0.000 1.000 1.000 1.000
2 0.339 0.980 0.965 0.000 1.000 1.000 1.000
1 0.988 1.000 1.000 0.412 1.000 1.000 1.000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 4. 구매유형, 거주지역별 평균 총 매출액
pd.pivot_table(df1, index='구매유형',columns='거주지역',values='총_매출액', aggfunc=np.mean)

거주지역,1,2,3,4,5,6,7
구매유형,,,,,,,
1,nan,3281350.000,3002825.000,3041428.182,3637031.667,3712146.667,3864880.000
2,nan,nan,4215648.571,8278686.562,6590330.674,10226770.763,13513839.437
3,4906400.000,3215055.000,6809777.143,7965439.677,11046081.250,13768678.750,25451441.176
4,nan,4034175.556,3720989.697,4229915.000,4272964.468,4749139.200,5138468.276


- 결과 해석
1. 등분산 검정
  + 두 변수 모두 귀무가설 기각(등분산이 아니다)??
2. 세 귀무가설 모두 유의확률 0.01 이하로 기각
3. 사후분석
  + 구매유형: 1-4 제외 모든 유형 간 유의
  + 거주지역: 3-6, 4-6, 5-6, 2-7, 3-7, 4-7, 5-7, 6-7 그룹간 총 매출액 차이 유의
4. 구매유형, 거주지역별 평균 총 매출액

  => 매출액은 구매유형에 따라, 거주지역에 따라 달라진다
  
  => 구매유형과 거주지역 간의 상호작용 효과 또한 유의하다
  
  => 특정 구매유형과 거주지역의 경우 다른 영역과는 다른 매출액 차이를 보일 것이다
    
  => 특히 거주지역(7), 구매유형(3)인 영역은 가장 큰 매출액 현황 보이고 있으므로  
    해당 고객군을 위한 차별화 마케팅 전략 수립 필요